In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
sampled_data=pd.read_csv('sampled_data.csv')

In [3]:
sampled_data.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,...,review_creation_date,review_answer_timestamp,seller_zip_code_prefix,seller_city,seller_state,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,e5535466d23681cc97013097a0644150,10d2967d139449709e9f095b22b0f1d0,delivered,2017-12-13 16:02:41,2017-12-15 13:15:29,2017-12-18 16:14:38,2018-01-24 17:22:31,2018-01-15 00:00:00,ef27ccb46b9ea8056a968d34a0b03374,28950,...,2018-01-19 00:00:00,2018-01-19 11:01:02,14840,guariba,SP,28950.0,-22.770718,-41.911736,armação dos búzios,RJ
1,8f78b315dd28cdac3c6fc9b52cb56795,aa22daee92f1c49865df08f7b28b4d57,delivered,2017-06-04 21:52:53,2017-06-04 22:02:25,2017-06-06 11:17:53,2017-06-07 14:04:53,2017-06-19 00:00:00,b7487f6dfc6da31a900f80971ffe8896,31720,...,2017-06-21 00:00:00,2017-06-21 20:52:49,31255,belo horizonte,MG,31720.0,-19.840435,-43.950514,belo horizonte,MG
2,366b94daa22781675a0ffffbcbd517a9,e3b2363f99d58cd6fe161ffe0ee40a95,delivered,2018-02-24 16:23:28,2018-02-24 16:30:55,2018-02-26 23:37:25,2018-03-06 18:11:38,2018-03-16 00:00:00,5cfeedd15a7c79f117a1a2cc2451cb4a,22230,...,2018-03-07 00:00:00,2018-03-09 17:50:58,13076,campinas,SP,22230.0,-22.937798,-43.177770,rio de janeiro,RJ
3,9ec41066fe9773c225c01e1a10bee705,88d28eeeb569b3f9acf4a1201b8cc38f,delivered,2018-02-28 21:31:09,2018-02-28 22:07:42,2018-03-05 10:42:53,2018-03-26 15:51:44,2018-04-03 00:00:00,8e6778e57f11d27fdad9007241a86d6d,15400,...,2018-03-27 00:00:00,2018-03-29 15:16:47,11704,praia grande,SP,15400.0,-20.740221,-48.888824,olímpia,SP
4,cebd81d91393b014d64df92a97a7f80c,862d78ca89647c96068b98673b2de19d,delivered,2018-07-30 16:17:23,2018-07-30 20:30:57,2018-07-31 15:18:00,2018-08-01 11:06:34,2018-08-02 00:00:00,8409538dd13716baf525432c99d7fb79,13482,...,2018-08-02 00:00:00,2018-08-05 23:38:21,9015,santo andre,SP,13482.0,-22.574867,-47.443778,limeira,SP


In [4]:
sampled_data.shape

(120998, 44)

<h1>
<span style ="color:blue">Setting Up Hypotheses for A/B Testing</span></h1>

<h3><span style ="color:black">Null Hypothesis (𝐻0): There is no significant difference in conversion rates for high-value orders between the treatment group (top categories) and the control group (other categories).






Alternative Hypothesis (H1): There is a significant difference in conversion rates for high-value orders between the treatment group (top categories) and the control group (other categories)</h3></span>

 <h4>A/B Test Setup</h4>


In [5]:
top_categories = sampled_data['product_category_name'].value_counts().head(10)
print(top_categories)

product_category_name
cama_mesa_banho           12816
beleza_saude               9765
moveis_decoracao           9423
esporte_lazer              9250
informatica_acessorios     8415
utilidades_domesticas      7802
relogios_presentes         5975
ferramentas_jardim         4832
brinquedos                 4426
telefonia                  4329
Name: count, dtype: int64


In [6]:
import numpy as np

# group assignment
sampled_data['group'] = np.where(sampled_data['product_category_name'].isin(top_categories.index), 
                                  'treatment', 'control')




treatment_group = sampled_data[sampled_data['group'] == 'treatment']
control_group = sampled_data[sampled_data['group'] == 'control']

# Check the size of each group
print(f"Treatment Group Size: {treatment_group.shape[0]}")
print(f"Control Group Size: {control_group.shape[0]}")


Treatment Group Size: 77033
Control Group Size: 43965


Calculating the Minimum Detectable Effect (MDE)

In [7]:
import numpy as np
from scipy.stats import norm


MDE= sqrt((p1(1−p1)/n)+(p2(1−p2)/n))⋅(zα+zβ)

<h2>Hypothesis Testing</h2>

In [8]:
sampled_data['Total_price']=sampled_data['price']+sampled_data['freight_value']

In [9]:
sampled_data['Total_price'].shape,sampled_data['price'].shape

((120998,), (120998,))

In [10]:
sampled_data['Total_price'].describe()

count    120998.000000
mean        139.458035
std         188.383992
min           6.080000
25%          55.952500
50%          92.300000
75%         157.130000
max        6922.210000
Name: Total_price, dtype: float64

In [11]:
# Selecting relevant numerical columns for summary statistics
numerical_columns = ['price', 'freight_value','Total_price', 'payment_value']  
summary_statistics = sampled_data[numerical_columns].describe()
print(summary_statistics)

# Summary for categorical columns
categorical_columns = ['product_category_name', 'payment_type']  
print('categorical_summary')
print(categorical_summary)



               price  freight_value    Total_price  payment_value
count  120998.000000  120998.000000  120998.000000  120986.000000
mean      119.676328      19.781708     139.458035     172.147484
std       181.368846      15.202120     188.383992     257.307260
min         0.850000       0.000000       6.080000       0.000000
25%        39.990000      13.530000      55.952500      61.730000
50%        74.990000      16.320000      92.300000     109.600000
75%       134.900000      20.710000     157.130000     189.830000
max      6729.000000     375.280000    6922.210000   13664.080000
categorical_summary
       product_category_name payment_type
count                 120998       120986
unique                    74            4
top          cama_mesa_banho  credit_card
freq                   12816        89144


Key Statistics:
The mean total price is around 139.46 units, and the mean payment value is 172.15 units. This suggests that, on average, the payment value exceeds the total price by about 23.4%.
However, this is based on the entire dataset, including outliers, so a 15% threshold seems practical and more conservative for defining "successful" orders.



top categories analysis

In [12]:
def calculate_metrics_top_categories(df, high_value_threshold, top_categories):
    # data to include only the top categories
    df_top = df[df['product_category_name'].isin(top_categories.index)]
    
    # 1. Order Delivered + Payment Confirmed
    successful_orders = df_top[(df_top['order_status'] == 'delivered') & ((df_top['payment_value'] - df_top['Total_price'])/df_top['Total_price'] >= 0.15 )]
    total_orders = df_top['order_id'].nunique()
    conversion_rate_delivered_payment_confirmed = successful_orders['order_id'].nunique() / total_orders if total_orders > 0 else 0

    # 2. High-Value Orders - ADJUSTED
    high_value_orders = df_top[df_top['payment_value'] >= high_value_threshold]
    rate_high_value_orders = high_value_orders['order_id'].nunique() / total_orders if total_orders > 0 else 0

    # 3. Category-Specific Conversion for Top Categories
    category_conversion_rates = df_top.groupby('product_category_name').apply(
        lambda x: (x[(x['order_status'] == 'delivered') & ((x['payment_value'] - x['Total_price'])/x['Total_price'] >= 0.15 )& (x['payment_value']>high_value_threshold)]['order_id'].nunique() / x['order_id'].nunique()) * 100 if x['order_id'].nunique() > 0 else 0
    )

    return {
        "Conversion Rate (Order Delivered + Payment Confirmed)": conversion_rate_delivered_payment_confirmed,
        " % (High-Value Orders)": rate_high_value_orders,
        "Category-Specific Conversion Rates (Top Categories)": category_conversion_rates
    }


high_value_threshold = 189  

# metrics
metrics_top_categories = calculate_metrics_top_categories(sampled_data, high_value_threshold, top_categories)

# results
for key, value in metrics_top_categories.items():
    if isinstance(value, pd.Series):  # Check if it's a Series for category-specific rates
        print(f"{key}:\n{value}\n")
    else:
        print(f"{key}: {value:.2%}")


Conversion Rate (Order Delivered + Payment Confirmed): 15.08%
 % (High-Value Orders): 22.18%
Category-Specific Conversion Rates (Top Categories):
product_category_name
beleza_saude               4.085646
brinquedos                 2.920057
cama_mesa_banho            9.806500
esporte_lazer              5.911920
ferramentas_jardim         9.822785
informatica_acessorios     9.719222
moveis_decoracao          10.736445
relogios_presentes         3.423485
telefonia                  2.294455
utilidades_domesticas      7.017011
dtype: float64



C:\Users\chatu\AppData\Local\Temp\ipykernel_18272\752789168.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  category_conversion_rates = df_top.groupby('product_category_name').apply(


In [13]:
# function to calculate conversion rates for treatment and control groups
def calculate_group_conversion_rate(df, top_categories):
    
    total_orders = df['order_id'].nunique()  # Total unique orders in this group
    
    # success condition
    converted_orders = df[(df['order_status'] == 'delivered') & 
                          ((df['payment_value'] - df['Total_price']) / df['Total_price'] > 0.15) & 
                          (df['payment_value'] > 189)]
    
    # number of successfully converted orders
    converted_orders_count = converted_orders['order_id'].nunique()  # Count of successful orders
    
    # Calculating the conversion rate
    conversion_rate = converted_orders_count / total_orders if total_orders > 0 else 0
    return conversion_rate

#  treatment and control groups based on the new definition
treatment_group = sampled_data[sampled_data['product_category_name'].isin(top_categories.index)]
control_group = sampled_data[~sampled_data['product_category_name'].isin(top_categories.index)]

# Calculate conversion rates for both groups
treatment_conversion_rate = calculate_group_conversion_rate(treatment_group, top_categories)
control_conversion_rate = calculate_group_conversion_rate(control_group, top_categories)

# Print the results
print(f"Treatment Group Conversion Rate: {treatment_conversion_rate:.2%}")
print(f"Control Group Conversion Rate: {control_conversion_rate:.2%}")


Treatment Group Conversion Rate: 6.79%
Control Group Conversion Rate: 5.29%


MDE: Minimum Detectable Effect (2% = 0.02)
p pooled: Pooled conversion rate, the average of control and treatment conversion rates.
Here’s what the formula represents:

Z-scores represent the required confidence and statistical power.
Pooled conversion rate averages the control and treatment group rates.
MDE reflects the desired detectable difference.
Now, based on these parameters and given that:

Baseline Conversion Rate (p1): 6.54% (or 0.0654),
MDE: 2% (0.02),
Power: 0.8,
Significance Level (alpha): 0.05,

<h3>Business Goal:
Objective: Detect a 2% increase in conversion rate for key product categories, especially high-value orders.</h3>

In [14]:
import numpy as np
from scipy.stats import norm

# parameters
alpha = 0.05  # Significance level
power = 0.8  # Desired power
baseline_conversion = 0.052  # Control group conversion rate
mde = 0.02  # Minimum Detectable Effect (2% increase in conversion rate)

# z-scores
z_alpha = norm.ppf(1 - alpha / 2)  # Two-tailed test
z_beta = norm.ppf(power)

# Pooled conversion rate (average of control and treatment groups)
pooled_conversion_rate = (baseline_conversion + (baseline_conversion + mde)) / 2

#sample size per group
sample_size_per_group = ((z_alpha + z_beta) ** 2 * pooled_conversion_rate * (1 - pooled_conversion_rate)) / (mde ** 2)
sample_size=np.ceil(sample_size_per_group)
print(f"Required sample size per group: {sample_size}")


Required sample size per group: 1142.0


In [15]:
import numpy as np
from scipy.stats import norm

# parameters
alpha = 0.05  # significance level
power = 0.8  # desired power
p1 = 0.0529  # control group conversion rate (5.29%%)
n = sample_size  # sample size per group
mde = 0.02  # minimum detectable effect (2%)

# z-scores
z_alpha = norm.ppf(1 - alpha / 2)  # two-tailed test
z_beta = norm.ppf(power)

# p2 (treatment group conversion rate)
p2 = p1 + mde

# p2 does not exceed 1
if p2 > 1:
    raise ValueError("The treatment group conversion rate exceeds 1.")

# MDE using the formula
MDE_value = np.sqrt((p1 * (1 - p1) / n) + (p2 * (1 - p2) / n)) * (z_alpha + z_beta)

print(f'Minimum Detectable Effect (MDE): {MDE_value:.4f}')


Minimum Detectable Effect (MDE): 0.0284


Conducting the A/B Test



 **Finding the Critical Value**:
   For a two-tailed test, divide the significance level \( \alpha \) by 2 and looking up the corresponding z-value from the standard normal distribution table. For \( \alpha = 0.05 \), the critical z-value is **1.96** (since 95% of the normal distribution lies within ±1.96 standard deviations from the mean).

 **Calculating the P-value**:
   The **p-value** is the probability that the observed result (or one more extreme) would occur if the null hypothesis were true.

   - If the absolute value of the z-statistic exceeds the critical z-value (e.g., \( |z| > 1.96 ### Explanation of the Two-Tailed Z-Test

A **two-tailed z-test** is a statistical test used to determine whether there is a significant difference between the means (or proportions) of two groups (e.g., treatment and control groups) in either direction—whether the treatment increases or decreases the metric being tested.

#### Steps in Conducting a Two-Tailed Z-Test

1. **Define Hypotheses**:
   - **Null Hypothesis (\( H_0 \))**: There is no difference between the two groups,this would mean that there is **no difference in conversion rates** between the top product categories and the rest.
     - \( H_0: p_{treatment} = p_{control} \)
   - **Alternative Hypothesis (\( H_1 \))**: There is a difference between the two groups (in either direction).
     - \( H_1: p_{treatment} \neq p_{control} \)

2. **Setting the Significance Level (\( \alpha \))**:
   - This is the probability of rejecting the null hypothesis when it is true (Type I error). Common choices for \( alpha ) are **0.05** (5%) or **0.01** (1%). For A/B testing, 0.05 is typically used.
   
3. **Calculate the Z-Statistic**:
   The z-statistic measures how far the sample result is from the null hypothesis, in terms of standard deviations.

   \)), the p-value will be **less than \( \alpha \)**, meaning the result is statistically significant.
   - The smaller the p-value, the stronger the evidence against the null hypothesis.

6. **Interpret the Results**:
   - **If \( |z| > 1.96 \)** (or \( p \)-value < 0.05): Reject the null hypothesis. This means there is a statistically significant difference in conversion rates between the two groups.
   - **If \( |z| \leq 1.96 \)** (or \( p \)-value ≥ 0.05): Fail to reject the null hypothesis. This means there is no significant difference in conversion rates.

---

### Explaination with our Data

#### Hypotheses:
- \( H_0 \): There is no difference in conversion rates between top product categories (treatment) and the rest (control).
- \( H_1 \): There is a difference in conversion rates between the two groups.



### Summary of How the Two-Tailed Z-Test Works:
1. **Hypotheses**: Test for any difference in either direction.
2. **Calculate Z-Statistic**: Measures how far the observed result is from the null hypothesis.
3. **Compare with Critical Value**: For a two-tailed test with \( \alpha = 0.05 \), the critical z-value is 1.96.
4. **P-value**: If the p-value < 0.05, reject the null hypothesis.
5. **Interpretation**: A significant result means a meaningful difference between the groups, while a non-significant result means no evidence of a difference.



In [19]:
def calculate_group_conversion_details(df):
    # Total unique orders in the dataframe
    total_orders = df['order_id'].nunique()  # Total unique orders in the group
    
    # orders that meet the success condition
    converted_orders = df[(df['order_status'] == 'delivered') & 
                          ((df['payment_value'] - df['Total_price']) / df['Total_price'] > 0.15) & 
                          (df['payment_value'] > 189)]
    
    # Count of successful orders
    converted_orders_count = converted_orders['order_id'].nunique()
    
    # Conversion rate calculation
    conversion_rate = converted_orders_count / total_orders if total_orders > 0 else 0
    
    return total_orders, converted_orders_count, conversion_rate

# Separating the treatment and control groups
treatment_group = sampled_data[sampled_data['group'] == 'treatment']
control_group = sampled_data[sampled_data['group'] == 'control']

# conversion details for both groups
treatment_total, treatment_success_count, treatment_conversion_rate = calculate_group_conversion_details(treatment_group)
control_total, control_success_count, control_conversion_rate = calculate_group_conversion_details(control_group)

# Print the results
print(f"Treatment Group Total Orders: {treatment_total}, Successful Conversions: {treatment_success_count}, Conversion Rate: {treatment_conversion_rate:.2%}")
print(f"Control Group Total Orders: {control_total}, Successful Conversions: {control_success_count}, Conversion Rate: {control_conversion_rate:.2%}")

# Setting n1 and n2 for z-test
n1 = treatment_success_count  # Successful conversions in treatment
n2 = control_success_count     # Successful conversions in control
print(n1)
print(n2)


Treatment Group Total Orders: 33757, Successful Conversions: 2291, Conversion Rate: 6.79%
Control Group Total Orders: 19853, Successful Conversions: 1050, Conversion Rate: 5.29%
2291
1050


In [20]:
# Checking the total number of unique orders in both groups
treatment_sample_size = treatment_group['order_id'].nunique()
control_sample_size = control_group['order_id'].nunique()

# sample sizes for each group
print(f"Treatment Group Sample Size: {treatment_sample_size}")
print(f"Control Group Sample Size: {control_sample_size}")

# conversion rates calculated previously
treatment_conversion_rate = calculate_group_conversion_rate(treatment_group, top_categories)
control_conversion_rate = calculate_group_conversion_rate(control_group, top_categories)

# Print conversion rates
print(f"Treatment Group Conversion Rate: {treatment_conversion_rate:.2%}")
print(f"Control Group Conversion Rate: {control_conversion_rate:.2%}")


Treatment Group Sample Size: 33757
Control Group Sample Size: 19853
Treatment Group Conversion Rate: 6.79%
Control Group Conversion Rate: 5.29%


In [21]:
import numpy as np

# parameters
p1 = 0.0679  # Treatment group conversion rate
p2 = 0.0529  # Control group conversion rate
n1 = 2291    # Sample size for treatment group
n2 = 1050   # Sample size for control group

# number of successes in each group
x1 = p1 * n1  # Conversions in treatment group
x2 = p2 * n2  # Conversions in control group

# pooled proportion
p_pool = (x1 + x2) / (n1 + n2)

# Z statistic
z_manual = (p1 - p2) / np.sqrt(p_pool * (1 - p_pool) * (1/n1 + 1/n2))

# p-value for two-tailed test
p_value_manual = 2 * (1 - norm.cdf(abs(z_manual)))

print(f"Z-Statistic: {z_manual:.4f}")
print(f"P-Value: {p_value_manual:.4f}")


Z-Statistic: 1.6543
P-Value: 0.0981


Interpretation:
Z-Statistic: A Z-statistic of 1.6543 shows that the difference between the conversion rates of the treatment and control groups is 1.65 standard deviations away from the expected difference under the null hypothesis (i.e., no difference between the two groups).

P-Value: The p-value of 0.0981 indicates the probability of observing such a difference in conversion rates (or a more extreme one) if the null hypothesis were true. Since the p-value is greater than the common significance level of 0.05 (or 5%), we fail to reject the null hypothesis.

Conclusion:
There isn't sufficient statistical evidence to say that the conversion rates between the treatment and control groups are significantly different, considering the 5% significance level (α=0.05). However, the result is borderline at a 10% significance level (α=0.10), 
